In [ ]:
!pip install memory-profiler

In [ ]:
import cv2
import numpy as np
import psutil
import time
from skimage.metrics import peak_signal_noise_ratio as psnr
import os

base_path = "/content/drive/My Drive/dataset"

def print_cpu_metrics(total_inference_time, num_images):
    cpu_percent = psutil.cpu_percent()
    memory_info = psutil.virtual_memory()
    avg_psnr = total_psnr / num_images if num_images > 0 else 0
    avg_inference_time = total_inference_time / num_images if num_images > 0 else 0
    print(f"Total CPU Usage: {cpu_percent}%")
    print(f"Total RAM Usage: {memory_info.used / (1024 ** 2):.2f} MB")
    print(f"Total Inference Time: {total_inference_time:.2f} ms")
    print(f"Average Inference Time per Image: {avg_inference_time:.2f} ms")
    print(f"Average PSNR: {avg_psnr:.2f} dB")

def non_local_means_denoising(image, h=10, hForColorComponents=10, templateWindowSize=7, searchWindowSize=21):

    start_time = time.time()
    if len(image.shape) == 2:
        denoised_image = cv2.fastNlMeansDenoising(image, None, h, templateWindowSize, searchWindowSize)
    else:
        denoised_image = cv2.fastNlMeansDenoisingColored(image, None, h, hForColorComponents, templateWindowSize, searchWindowSize)
    end_time = time.time()
    inference_time = (end_time - start_time) * 1000
    return denoised_image, inference_time

def calculate_psnr(original_image, denoised_image):
    return psnr(original_image, denoised_image)

def process_folder(input_folder, output_folder):
    input_path = os.path.join(base_path, input_folder)
    output_path = os.path.join(base_path, output_folder)

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_files = [f for f in os.listdir(input_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    folder_inference_time = 0
    folder_psnr_sum = 0
    num_images = len(image_files)

    for image_file in image_files:
        image_path = os.path.join(input_path, image_file)
        image = cv2.imread(image_path)

        if image.shape[0] > 720 or image.shape[1] > 1280:
            raise ValueError("Input image size exceeds 1280x720 resolution")

        denoised_image, inference_time = non_local_means_denoising(image)

        psnr_value = calculate_psnr(image, denoised_image)
        folder_psnr_sum += psnr_value
        folder_inference_time += inference_time

        output_file_path = os.path.join(output_path, f"denoised_{image_file}")
        cv2.imwrite(output_file_path, denoised_image)

    return folder_inference_time, folder_psnr_sum, num_images

total_inference_time = 0
total_psnr = 0
total_images = 0

for folder in ['noisy5', 'noisy15', 'noisy35']:
    output_folder = f"denoised{folder[-2:]}"
    inference_time, psnr_sum, num_images = process_folder(folder, output_folder)
    total_inference_time += inference_time
    total_psnr += psnr_sum
    total_images += num_images

print_cpu_metrics(total_inference_time, total_images)